<a href="https://colab.research.google.com/github/fcoliveira-utfpr/agrometeorologia/blob/main/01_dados_nasapower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Início - Bibliotecas**
---

In [16]:
#importando bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
import requests
import os
import openpyxl
from google.colab import files
import matplotlib.pyplot as plt
from itertools import repeat

# **Ver ESTADOS e MUNICÍPIOS**
---

In [ ]:
# Escolha de estado e município
url_42 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df42 = pd.read_csv(url_42)

estados = sorted(df42['Estado'].unique())

estado_dropdown = widgets.Dropdown(
    options=estados,
    description='Estado:',
    layout=widgets.Layout(width='250px')
)

municipio_dropdown = widgets.Dropdown(
    description='Município:',
    layout=widgets.Layout(width='350px')
)
def atualizar_municipios(change):
    estado_sel = change['new']
    municipios = sorted(
        df42[df42['Estado'] == estado_sel]['Município'].unique()
    )
    municipio_dropdown.options = municipios

estado_dropdown.observe(atualizar_municipios, names='value')

estado_dropdown.value = estados[0]

display(estado_dropdown, municipio_dropdown)

# **Baixar dados NASA/POWER diários**
---

In [9]:
# ============================================================
# Download de dados NASA POWER
# ============================================================

municipio = "Cascavel" ###### Nome aqui entre aspas

estado = "PR" ###### Sigla do estado entre aspas

# Define período de coleta
data_ini = "12/05/2022"   ## Data de início dos dados no formato "dia/mês/ano"
data_fim = "12/06/2022"   ## Data de fim dos dados no formato "dia/mês/ano"

# ============================================================
# ############################################################
# ============================================================

# Formata datas para API
ini = int(datetime.strptime(data_ini, '%d/%m/%Y').strftime('%Y%m%d'))
fin = int(datetime.strptime(data_fim, '%d/%m/%Y').strftime('%Y%m%d'))

url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)

# Separa colunas categóricas
cidade = df1['Município']
uf = df1['Estado']
regiao = df1['Região']
koppen = df1['Köppen']

# Converte valores com vírgula para ponto e numérico
df1_valores = df1.drop(columns=['Município', 'Estado','Região','Köppen'])
df1_valores = df1_valores.replace({',': '.'}, regex=True)
df1_valores = df1_valores.apply(pd.to_numeric, errors='coerce')

# Reconstrói o df
df1 = df1_valores.copy()
df1['Município'] = cidade
df1['Estado'] = uf
df1['Região'] = regiao
df1['Köppen'] = koppen

# Filtra município e estado
df11 = df1[(df1['Município'] == municipio) & (df1['Estado'] == estado)].copy()
# Metadados do local
nome_cidade = df11["Município"].iloc[0]
latitude = df11["latitude"].iloc[0]
longitude = df11["longitude"].iloc[0]
alt = df11["Altitude"].iloc[0]

# Monta URL da requisição
base_url = (
    "https://power.larc.nasa.gov/api/temporal/daily/point"
    "?parameters=T2M_MIN,T2M_MAX,PRECTOTCORR,RH2M,WS2M,ALLSKY_SFC_SW_DWN,TOA_SW_DWN"
    "&community=RE"
    "&longitude={longitude}"
    "&latitude={latitude}"
    "&start={ini}"
    "&end={fin}"
    "&format=JSON"
)

api_request_url = base_url.format(
    longitude=longitude,
    latitude=latitude,
    ini=ini,
    fin=fin
)

# Requisição
response = requests.get(url=api_request_url, verify=True, timeout=120)
response.raise_for_status()

# Extração dos parâmetros
dados_json = response.json()
propriedades = dados_json['properties']
parametros = propriedades['parameter']

# Datas
datas_str = sorted(list(parametros['T2M_MIN'].keys()))
datas = [pd.to_datetime(d) for d in datas_str]

# Constrói DataFrame com variáveis
df = pd.DataFrame({'Data': datas})
df['Tmin'] = [parametros['T2M_MIN'][d] for d in datas_str]
df['Tmax'] = [parametros['T2M_MAX'][d] for d in datas_str]
df['Chuva'] = [parametros['PRECTOTCORR'][d] for d in datas_str]
df['UR'] = [parametros['RH2M'][d] for d in datas_str]
df['U2'] = [parametros['WS2M'][d] for d in datas_str]
df['Rs_raw'] = [parametros['ALLSKY_SFC_SW_DWN'][d] for d in datas_str]
df['Qo_raw'] = [parametros['TOA_SW_DWN'][d] for d in datas_str]

# Converte radiação para MJ/m²
df['Rs'] = df['Rs_raw'] * 3.6
df['Qo'] = df['Qo_raw'] * 3.6

# Variáveis intermediárias
df['Tmed'] = (df['Tmax'] + df['Tmin']) / 2
df['Patm'] = 101.3 * ((293 - 0.0065 * alt) / 293) ** 5.26
df['NDA'] = [pd.to_datetime(d).timetuple().tm_yday for d in df['Data']]

# Declinação solar
def dec_sol(NDA):
    return 23.45 * np.sin(np.deg2rad(360/365 * (NDA - 80)))

df['d'] = df['NDA'].apply(dec_sol)

# Hora de nascer do sol
def hora_na_sol(d, Lat):
    return np.rad2deg(np.arccos(-(np.tan(np.deg2rad(Lat)) * np.tan(np.deg2rad(d)))))

df['Hn'] = df['d'].apply(lambda d_: hora_na_sol(d_, latitude))


# (d/D)^2
def relacao_d_D_2(NDA):
    return 1 + 0.033 * np.cos(np.deg2rad(NDA * 360/365))
df['(d/D)²'] = df['NDA'].apply(relacao_d_D_2)

# Balanços radiativos
df['BOC'] = df['Rs'] * 0.75  # (1 - 25%)

df['N'] = df.apply(lambda x: (2 * x['Hn']) / 15, axis = 1)

# Calcular n, mas limitando ao valor de N
df['n'] = df.apply(
    lambda row: min(
        (row['N']/0.52) * (0.16 * np.sqrt(row['Tmax'] - row['Tmin']) - 0.29 * np.cos(np.deg2rad(latitude))),
        row['N']
    ),
    axis=1
)
def e_saturacao(temp):
    return 0.6108 * 10 ** ((7.5 * temp) / (237.3 + temp))

df['es_Tmax'] = df['Tmax'].apply(e_saturacao)
df['es_Tmin'] = df['Tmin'].apply(e_saturacao)
df['es'] = (df['es_Tmax'] + df['es_Tmin']) / 2
df['ea'] = (df['UR']/100) * df['es']
df['QGcs'] = df['Qo'] * (0.75 + (2e-5)*alt)

def BOL(tmax, tmin, eaa, qg, qgcs):
    a = 4.903e-9 * (((tmax+273.16)**4 + (tmin+273.16)**4)/2)
    b = (0.34 - 0.14*np.sqrt(eaa))
    c = 1.35*(qg/qgcs) - 0.35
    return -(a*b*c)

df['BOL'] = df.apply(lambda x: BOL(x['Tmax'],x['Tmin'],x['ea'],x['Rs'],x['QGcs']), axis=1)
df['Rn'] = df['BOC'] + df['BOL']

df['s'] = (4098 * df['es']) / (df['Tmed'] + 237.3)**2
df['gama'] = 0.665e-3 * df['Patm']

# Penman-Monteith
def ETo(s,Rn,gama,u2,es,ea,tmed):
    ETo1 = 0.408*s*Rn
    ETo2 = (gama*900*u2*(es-ea))/(tmed+273)
    ETo3 = s + gama*(1+0.34*u2)
    return (ETo1 + ETo2)/ETo3

df['ETo'] = df.apply(lambda x: ETo(x['s'],x['Rn'],x['gama'],x['U2'],x['es'],x['ea'],x['Tmed']), axis=1)

# Metadados no df
df['Municipio'] = municipio
df['Lat'] = latitude
df['Lon'] = longitude
df['Alt'] = alt

tiempo = 'diarios'

#para baixar os dados tire o # da última linha
from google.colab import files
df.to_excel(f'{municipio}_{tiempo}.xlsx')
#files.download(f'{municipio}_{tiempo}.xlsx')

df

,Data,Tmin,Tmax,Chuva,UR,U2,Rs_raw,Qo_raw,Rs,Qo,...,QGcs,BOL,Rn,s,gama,ETo,Municipio,Lat,Lon,Alt
0,2022-05-12,10.04,20.96,0.02,82.41,0.15,4.1554,6.6876,14.95944,24.07536,...,18.374796,-4.263888,6.955692,0.119014,0.062328,1.882028,Cascavel,-25.029371,-53.379713,661
1,2022-05-13,10.29,21.92,0.05,84.33,0.11,4.7873,6.6475,17.23428,23.93100,...,18.264618,-5.113919,7.811791,0.123900,0.062328,2.128712,Cascavel,-25.029371,-53.379713,661
2,2022-05-14,11.73,23.30,2.38,86.23,0.07,0.7176,6.6082,2.58336,23.78952,...,18.156637,0.835231,2.772751,0.133750,0.062328,0.785848,Cascavel,-25.029371,-53.379713,661
3,2022-05-15,12.95,21.72,2.43,87.60,0.04,2.0287,6.5698,7.30332,23.65128,...,18.051130,-1.046623,4.430867,0.129308,0.062328,1.224636,Cascavel,-25.029371,-53.379713,661
4,2022-05-16,6.90,17.00,0.05,72.77,0.05,4.6152,6.5321,16.61472,23.51556,...,17.947546,-5.706057,6.754983,0.096723,0.062328,1.689466,Cascavel,-25.029371,-53.379713,661
5,2022-05-17,5.72,12.12,0.02,71.87,0.05,2.5721,6.4951,9.25956,23.38236,...,17.845885,-2.306787,4.637883,0.078777,0.062328,1.071484,Cascavel,-25.029371,-53.379713,661
6,2022-05-18,6.15,10.19,0.04,85.33,0.05,1.4237,6.4570,5.12532,23.24520,...,17.741202,-0.251621,3.592369,0.074420,0.062328,0.803133,Cascavel,-25.029371,-53.379713,661
7,2022-05-19,5.72,15.67,0.02,80.86,0.06,2.7533,6.4205,9.91188,23.11380,...,17.640914,-2.524736,4.909174,0.089866,0.062328,1.192834,Cascavel,-25.029371,-53.379713,661
8,2022-05-20,4.49,17.26,0.00,78.42,0.32,4.6418,6.3854,16.71048,22.98744,...,17.544474,-5.781378,6.751482,0.093534,0.062328,1.702079,Cascavel,-25.029371,-53.379713,661
9,2022-05-21,3.96,18.63,0.02,71.58,0.31,4.1354,6.3521,14.88744,22.86756,...,17.452979,-5.062029,6.103551,0.098076,0.062328,1.616690,Cascavel,-25.029371,-53.379713,661


# **Baixar dados NASA/POWER mensal**
---

In [19]:
# ============================================================
# Download de dados NASA POWER
# ============================================================

municipio = "Dois Vizinhos" ###### Nome aqui entre aspas

estado = "PR" ###### Sigla do estado entre aspas

# Define período de coleta
ano_ini = "2022"   ## Ano de início dos dados no formato "ano"
ano_fim = "2023"   ## Data de fim dos dados no formato "ano"

# ============================================================
# ############################################################
# ============================================================
base_dados = 'NP'
url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)

# Separa colunas categóricas
cidade = df1['Município']
uf = df1['Estado']
regiao = df1['Região']
koppen = df1['Köppen']

# Converte valores com vírgula para ponto e numérico
df1_valores = df1.drop(columns=['Município', 'Estado','Região','Köppen'])
df1_valores = df1_valores.replace({',': '.'}, regex=True)
df1_valores = df1_valores.apply(pd.to_numeric, errors='coerce')

# Reconstrói o df
df1 = df1_valores.copy()
df1['Município'] = cidade
df1['Estado'] = uf
df1['Região'] = regiao
df1['Köppen'] = koppen

# Filtra município e estado
df11 = df1[(df1['Município'] == municipio) & (df1['Estado'] == estado)].copy()
# Metadados do local
nome_cidade = df11["Município"].iloc[0]
latitude = df11["latitude"].iloc[0]
longitude = df11["longitude"].iloc[0]
alt = df11["Altitude"].iloc[0]

ais = datetime.strptime(ano_ini, '%Y').date()
ini = int(ais.strftime('%Y'))
afs = datetime.strptime(ano_fim , '%Y').date()
fin = int(afs.strftime('%Y'))

dados = [(latitude, longitude, ini, fin)]
output = r""
base_url = (
    r"https://power.larc.nasa.gov/api/temporal/monthly/point?"
    r"parameters=ALLSKY_SFC_SW_DWN,TOA_SW_DWN,WS2M,T2M_MIN,T2M_MAX,T2M,RH2M,PS,PRECTOTCORR,PRECTOTCORR_SUM,CLRSKY_SFC_SW_DWN&"
    r"community=RE&longitude={longitude}&latitude={latitude}&"
    r"start={ini}&end={fin}&format=JSON"
)
for latitude, longitude, ini, fin in dados:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude, ini=ini, fin=fin)
    response = requests.get(url=api_request_url, verify=True, timeout=120)
json = response.json()
parametros = json['properties']

def extract_parameter_df(parametros, parameter_name, column_name):
    param_df = pd.DataFrame.from_dict(parametros, orient='index')
    param_values = param_df[parameter_name].to_list()
    df_param = pd.DataFrame(param_values).T
    df_param = df_param.rename(columns={0: column_name}, inplace=False)
    return df_param

def create_parameter_df(parametros, parameter_name, column_name):
    df_param = extract_parameter_df(parametros, parameter_name, column_name)
    return pd.DataFrame({f'{column_name} ({parameter_name})': df_param[column_name].to_list()})

N_anos = fin - ini+1
anos = list(range(ini, fin+1))
Anos = [item for item in anos for _ in repeat(None, 13)]
periodo = list(range(1, 14)) * N_anos

df = pd.DataFrame({'Mês': periodo})
df['Anos'] = Anos
df = df.join(create_parameter_df(parametros, 'T2M_MAX', 'Tmax'))
df = df.join(create_parameter_df(parametros, 'T2M_MIN', 'Tmin'))
df = df.join(create_parameter_df(parametros, 'T2M', 'T'))
df = df.join(create_parameter_df(parametros, 'PRECTOTCORR', 'Chuva (mm/d)'))
df = df.join(create_parameter_df(parametros, 'PRECTOTCORR_SUM', 'Chuva (mm)'))
df = df.join(create_parameter_df(parametros, 'RH2M', 'UR'))
df = df.join(create_parameter_df(parametros, 'WS2M', 'U2'))
df = df.join(create_parameter_df(parametros, 'ALLSKY_SFC_SW_DWN', 'Rs'))
df = df.join(create_parameter_df(parametros, 'CLRSKY_SFC_SW_DWN', 'Qgcs'))
df = df.join(create_parameter_df(parametros, 'TOA_SW_DWN', 'Qo'))
df = df.join(create_parameter_df(parametros, 'PS', 'Patm'))

#transformando a radiação solar em MJ/m²d
df['Qg (MJ/m²d)'] = df.apply(lambda x: x['Rs (ALLSKY_SFC_SW_DWN)'] * 3.6, axis=1)
df['Qo (MJ/m²d)'] = df.apply(lambda x: x['Qo (TOA_SW_DWN)'] * 3.6, axis=1)
df['QGcs'] = df.apply(lambda x: x['Qgcs (CLRSKY_SFC_SW_DWN)'] * 3.6, axis=1)
df['Tmed (°C)'] = df.apply(lambda x: (x['Tmax (T2M_MAX)'] + x['Tmin (T2M_MIN)']) / 2, axis=1)

# Filtrando as linhas onde o valor da coluna "Mês" é 13
df = df[~df['Mês'].isin([13])]

# Data dia 15 de cada mês:
df['Data'] = pd.to_datetime({
    'year': df['Anos'],
    'month': df['Mês'],
    'day': 15
})

df['NDA'] = [pd.to_datetime(d).timetuple().tm_yday for d in df['Data']]

# Declinação solar
def dec_sol(NDA):
    return 23.45 * np.sin(np.deg2rad(360/365 * (NDA - 80)))

df['d'] = df['NDA'].apply(dec_sol)

# Hora de nascer do sol
def hora_na_sol(d, Lat):
    return np.rad2deg(np.arccos(-(np.tan(np.deg2rad(Lat)) * np.tan(np.deg2rad(d)))))

df['Hn'] = df['d'].apply(lambda d_: hora_na_sol(d_, latitude))

df['N'] = df.apply(lambda x: (2 * x['Hn']) / 15, axis = 1)

# Calcular n, mas limitando ao valor de N
df['n'] = df.apply(
    lambda row: min(
        (row['N']/0.52) * (0.16 * np.sqrt(row['Tmax (T2M_MAX)'] - row['Tmin (T2M_MIN)']) - 0.29 * np.cos(np.deg2rad(latitude))),
        row['N']
    ),
    axis=1
)

#Balanço de ondas curtas - BOC
df['BOC'] = df.apply(lambda x: x['Qg (MJ/m²d)'] * (1 - 25 / 100), axis = 1)

#função para pressão de saturação e atual/parcial
def e(temp):
  return 0.6108 * 10 **((7.5*temp)/(237.3+temp))

#pressão de saturação para Tmax - esTmax (kPa)
df['es_Tmax'] = df.apply(lambda x: e(x['Tmax (T2M_MAX)']), axis = 1)

#pressão de saturação para Tmin - esTmin (kPa)
df['es_Tmin'] = df.apply(lambda x: e(x['Tmin (T2M_MIN)']), axis = 1)

#pressão de saturação es (kPa)
df['es'] = df.apply(lambda x: (x['es_Tmin'] + x['es_Tmax'])/2, axis = 1)

#pressão parcial de vapor ea
df['ea'] = df.apply(lambda x: (x['UR (RH2M)']/100) * x['es'], axis=1)

#determinando BOL
def BOL(tmax, tmin, eaa, qg, qgcs):
  a = (4.903 * (10**-9)*(((tmax**4)+(tmin**4))/2))
  b = (0.34 - 0.14 * np.sqrt(eaa))
  c = 1.35 * (qg/qgcs) - 0.35
  return -(a*b*c)

df['BOL'] = df.apply(lambda x: BOL(x['Tmax (T2M_MAX)'], x['Tmin (T2M_MIN)'], x['ea'], x['Qg (MJ/m²d)'], x['QGcs']), axis = 1)

#determinando Rn
df['Rn'] = df.apply(lambda x: x['BOC'] + x['BOL'], axis = 1)

#determinando s - declividade da curva de saturação
df['s'] = df.apply(lambda x: (4098 * x['es'])/(x['Tmed (°C)'] +237.3)**2, axis = 1)

#determinando gama - constante psicrométrica
df['gama'] = df.apply(lambda x: (0.665*10**-3) * x['Patm (PS)'], axis = 1)

#determinando ETo
def ETo(s, Rn, gama, u2, es, ea, tmed):
  ETo1 = 0.408 * s * Rn
  ETo2 = (gama * 900 * u2 * (es - ea))/(tmed + 273)
  ETo3 = s + gama * (1 + 0.34 * u2)
  return (ETo1 + ETo2) / ETo3

df['ETo'] = df.apply(lambda x: ETo(x['s'], x['Rn'], x['gama'], x['U2 (WS2M)'], x['es'], x['ea'], x['Tmed (°C)']), axis = 1)

# Função para determinar se um ano é bissexto
def is_leap_year(year):
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True
            else:
                return False
        else:
            return True
    else:
        return False

# Dicionário com o número de dias em cada mês
dias_por_mes = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

# Função para obter o número de dias de um mês em um determinado ano
def numero_dias_mes(mes, ano):
    if mes == 2 and is_leap_year(ano):
        return 29
    else:
        return dias_por_mes[mes]

# Aplicar a função para criar a nova coluna "N°dias"
df['N°dias'] = df.apply(lambda x: numero_dias_mes(x['Mês'], x['Anos']), axis=1)

# Calculando ETo mensal
df['ETo (mm/mês)'] = df.apply(lambda x: x['ETo'] * x['N°dias'], axis = 1)

# Calculando Rn mensal
df['Rn (MJ/m²mês)'] = df.apply(lambda x: x['Rn'] * x['N°dias'], axis = 1)

# Calculando n mensal
df['n (horas/mês)'] = df.apply(lambda x: x['n'] * x['N°dias'], axis = 1)

df['Municipio'] = df.apply(lambda x: nome_cidade, axis=1)

#selecionamdo as colunas de interesse
df = df[['Mês','Anos','Tmax (T2M_MAX)', 'Tmin (T2M_MIN)','Tmed (°C)', \
    'Chuva (mm) (PRECTOTCORR_SUM)','UR (RH2M)', 'U2 (WS2M)', \
         'Rn', 'Rn (MJ/m²mês)', \
         'N', 'n', \
         'ETo','ETo (mm/mês)','Municipio']]

# Renomear várias colunas diretamente
df.columns = ['Mês','Anos','Tmax (°C)', 'Tmin (°C)','Tmed (°C)', \
              'Chuva_total (mm)', 'UR_med (%)', 'U2_med (m/s)', \
              'Rn (MJ/m²d)', 'Rn (MJ/m²mês)', \
              'N', 'n', \
              'ETo (mm/d)','ETo (mm/mês)','Municipio']

tiempo = 'mensal'

#para baixar os dados tire o # da última linha
from google.colab import files
df.to_excel(f'{municipio}_{tiempo}.xlsx')
#files.download(f'{municipio}_{tiempo}.xlsx')

df

,Mês,Anos,Tmax (°C),Tmin (°C),Tmed (°C),Chuva_total (mm),UR_med (%),U2_med (m/s),Rn (MJ/m²d),Rn (MJ/m²mês),N,n,ETo (mm/d),ETo (mm/mês),Municipio
0,1,2022,39.41,16.21,27.810,108.81,67.95,0.07,17.614125,546.037888,13.429932,13.157717,5.834299,180.863257,Dois Vizinhos
1,2,2022,36.32,10.54,23.430,89.11,64.66,0.10,18.182990,509.123714,12.849391,12.849391,5.822921,163.041802,Dois Vizinhos
2,3,2022,33.96,10.09,22.025,260.81,83.01,0.07,13.470342,417.580614,12.155595,12.155595,4.186627,129.785422,Dois Vizinhos
3,4,2022,28.22,6.68,17.450,239.77,86.65,0.11,10.765847,322.975408,11.363893,10.519844,3.106294,93.188811,Dois Vizinhos
4,5,2022,25.55,3.02,14.285,179.21,86.13,0.15,8.522455,264.196101,10.723064,10.274558,2.350796,72.874686,Dois Vizinhos
5,6,2022,24.54,-1.50,11.520,88.73,87.85,0.13,8.305908,249.177250,10.397175,10.397175,2.221931,66.657931,Dois Vizinhos
6,7,2022,25.23,3.57,14.400,65.20,83.62,0.14,10.129995,314.029848,10.550678,9.808940,2.785340,86.345543,Dois Vizinhos
7,8,2022,26.09,-0.97,12.560,176.94,82.00,0.13,10.805268,334.963301,11.116439,11.116439,2.972510,92.147810,Dois Vizinhos
8,9,2022,27.23,2.65,14.940,188.46,83.48,0.08,9.920778,297.623352,11.883247,11.883247,2.799730,83.991905,Dois Vizinhos
9,10,2022,28.13,8.15,18.140,413.08,85.04,0.09,13.107034,406.318053,12.648280,11.042478,3.794670,117.634767,Dois Vizinhos


# **Baixar dados NASA/POWER Normal Climatológica**
---

In [21]:
# ============================================================
# Download de dados NASA POWER
# ============================================================

municipio = "Pato Branco" ###### Nome aqui entre aspas

estado = "PR" ###### Sigla do estado entre aspas

# Define período de coleta
# Normal Climatológica padrão NASA-POWER (1991-2020)
ano_ini = "1991"   ## Ano de início dos dados no formato "ano"
ano_fim = "2020"   ## Data de fim dos dados no formato "ano"

# ============================================================
# ############################################################
# ============================================================

url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)

# Separa colunas categóricas
cidade = df1['Município']
uf = df1['Estado']
regiao = df1['Região']
koppen = df1['Köppen']

# Converte valores com vírgula para ponto e numérico
df1_valores = df1.drop(columns=['Município', 'Estado','Região','Köppen'])
df1_valores = df1_valores.replace({',': '.'}, regex=True)
df1_valores = df1_valores.apply(pd.to_numeric, errors='coerce')

# Reconstrói o df
df1 = df1_valores.copy()
df1['Município'] = cidade
df1['Estado'] = uf
df1['Região'] = regiao
df1['Köppen'] = koppen

# Filtra município e estado
df11 = df1[(df1['Município'] == municipio) & (df1['Estado'] == estado)].copy()
# Metadados do local
nome_cidade = df11["Município"].iloc[0]
latitude = df11["latitude"].iloc[0]
longitude = df11["longitude"].iloc[0]
alt = df11["Altitude"].iloc[0]

ais = datetime.strptime(ano_ini, '%Y').date()
ini = int(ais.strftime('%Y'))
afs = datetime.strptime(ano_fim , '%Y').date()
fin = int(afs.strftime('%Y'))

dados = [(latitude, longitude, ini, fin)]
output = r""
base_url = (
    r"https://power.larc.nasa.gov/api/temporal/monthly/point?"
    r"parameters=ALLSKY_SFC_SW_DWN,TOA_SW_DWN,WS2M,T2M_MIN,T2M_MAX,T2M,RH2M,PS,PRECTOTCORR,PRECTOTCORR_SUM,CLRSKY_SFC_SW_DWN&"
    r"community=RE&longitude={longitude}&latitude={latitude}&"
    r"start={ini}&end={fin}&format=JSON"
)
for latitude, longitude, ini, fin in dados:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude, ini=ini, fin=fin)
    response = requests.get(url=api_request_url, verify=True, timeout=120)
json = response.json()
parametros = json['properties']

def extract_parameter_df(parametros, parameter_name, column_name):
    param_df = pd.DataFrame.from_dict(parametros, orient='index')
    param_values = param_df[parameter_name].to_list()
    df_param = pd.DataFrame(param_values).T
    df_param = df_param.rename(columns={0: column_name}, inplace=False)
    return df_param

def create_parameter_df(parametros, parameter_name, column_name):
    df_param = extract_parameter_df(parametros, parameter_name, column_name)
    return pd.DataFrame({f'{column_name} ({parameter_name})': df_param[column_name].to_list()})

N_anos = fin - ini+1
anos = list(range(ini, fin+1))
Anos = [item for item in anos for _ in repeat(None, 13)]
periodo = list(range(1, 14)) * N_anos

df = pd.DataFrame({'Mês': periodo})
df['Anos'] = Anos
df = df.join(create_parameter_df(parametros, 'T2M_MAX', 'Tmax'))
df = df.join(create_parameter_df(parametros, 'T2M_MIN', 'Tmin'))
df = df.join(create_parameter_df(parametros, 'T2M', 'T'))
df = df.join(create_parameter_df(parametros, 'PRECTOTCORR', 'Chuva (mm/d)'))
df = df.join(create_parameter_df(parametros, 'PRECTOTCORR_SUM', 'Chuva (mm)'))
df = df.join(create_parameter_df(parametros, 'RH2M', 'UR'))
df = df.join(create_parameter_df(parametros, 'WS2M', 'U2'))
df = df.join(create_parameter_df(parametros, 'ALLSKY_SFC_SW_DWN', 'Rs'))
df = df.join(create_parameter_df(parametros, 'CLRSKY_SFC_SW_DWN', 'Qgcs'))
df = df.join(create_parameter_df(parametros, 'TOA_SW_DWN', 'Qo'))
df = df.join(create_parameter_df(parametros, 'PS', 'Patm'))

#transformando a radiação solar em MJ/m²d
df['Qg (MJ/m²d)'] = df.apply(lambda x: x['Rs (ALLSKY_SFC_SW_DWN)'] * 3.6, axis=1)
df['Qo (MJ/m²d)'] = df.apply(lambda x: x['Qo (TOA_SW_DWN)'] * 3.6, axis=1)
df['QGcs'] = df.apply(lambda x: x['Qgcs (CLRSKY_SFC_SW_DWN)'] * 3.6, axis=1)
df['Tmed (°C)'] = df.apply(lambda x: (x['Tmax (T2M_MAX)'] + x['Tmin (T2M_MIN)']) / 2, axis=1)

# Filtrando as linhas onde o valor da coluna "Mês" é 13
df = df[~df['Mês'].isin([13])]

# Data dia 15 de cada mês:
df['Data'] = pd.to_datetime({
    'year': df['Anos'],
    'month': df['Mês'],
    'day': 15
})

df['NDA'] = [pd.to_datetime(d).timetuple().tm_yday for d in df['Data']]

# Declinação solar
def dec_sol(NDA):
    return 23.45 * np.sin(np.deg2rad(360/365 * (NDA - 80)))

df['d'] = df['NDA'].apply(dec_sol)

# Hora de nascer do sol
def hora_na_sol(d, Lat):
    return np.rad2deg(np.arccos(-(np.tan(np.deg2rad(Lat)) * np.tan(np.deg2rad(d)))))

df['Hn'] = df['d'].apply(lambda d_: hora_na_sol(d_, latitude))

df['N'] = df.apply(lambda x: (2 * x['Hn']) / 15, axis = 1)

# Calcular n, mas limitando ao valor de N
df['n'] = df.apply(
    lambda row: min(
        (row['N']/0.52) * (0.16 * np.sqrt(row['Tmax (T2M_MAX)'] - row['Tmin (T2M_MIN)']) - 0.29 * np.cos(np.deg2rad(latitude))),
        row['N']
    ),
    axis=1
)

#Balanço de ondas curtas - BOC
df['BOC'] = df.apply(lambda x: x['Qg (MJ/m²d)'] * (1 - 25 / 100), axis = 1)
#função para pressão de saturação e atual/parcial
def e(temp):
  return 0.6108 * 10 **((7.5*temp)/(237.3+temp))

#pressão de saturação para Tmax - esTmax (kPa)
df['es_Tmax'] = df.apply(lambda x: e(x['Tmax (T2M_MAX)']), axis = 1)

#pressão de saturação para Tmin - esTmin (kPa)
df['es_Tmin'] = df.apply(lambda x: e(x['Tmin (T2M_MIN)']), axis = 1)

#pressão de saturação es (kPa)
df['es'] = df.apply(lambda x: (x['es_Tmin'] + x['es_Tmax'])/2, axis = 1)

#pressão parcial de vapor ea
df['ea'] = df.apply(lambda x: (x['UR (RH2M)']/100) * x['es'], axis=1)

#determinando BOL
def BOL(tmax, tmin, eaa, qg, qgcs):
  a = (4.903 * (10**-9)*(((tmax**4)+(tmin**4))/2))
  b = (0.34 - 0.14 * np.sqrt(eaa))
  c = 1.35 * (qg/qgcs) - 0.35
  return -(a*b*c)

df['BOL'] = df.apply(lambda x: BOL(x['Tmax (T2M_MAX)'], x['Tmin (T2M_MIN)'], x['ea'], x['Qg (MJ/m²d)'], x['QGcs']), axis = 1)

#determinando Rn
df['Rn'] = df.apply(lambda x: x['BOC'] + x['BOL'], axis = 1)

# Calculando n mensal
df['n (horas/mês)'] = df.apply(lambda x: x['n'] * x['N°dias'], axis = 1)

#determinando s - declividade da curva de saturação
df['s'] = df.apply(lambda x: (4098 * x['es'])/(x['Tmed (°C)'] +237.3)**2, axis = 1)

#determinando gama - constante psicrométrica
df['gama'] = df.apply(lambda x: (0.665*10**-3) * x['Patm (PS)'], axis = 1)

#determinando ETo
def ETo(s, Rn, gama, u2, es, ea, tmed):
  ETo1 = 0.408 * s * Rn
  ETo2 = (gama * 900 * u2 * (es - ea))/(tmed + 273)
  ETo3 = s + gama * (1 + 0.34 * u2)
  return (ETo1 + ETo2) / ETo3

df['ETo'] = df.apply(lambda x: ETo(x['s'], x['Rn'], x['gama'], x['U2 (WS2M)'], x['es'], x['ea'], x['Tmed (°C)']), axis = 1)

# Função para determinar se um ano é bissexto
def is_leap_year(year):
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True
            else:
                return False
        else:
            return True
    else:
        return False

# Dicionário com o número de dias em cada mês
dias_por_mes = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

# Função para obter o número de dias de um mês em um determinado ano
def numero_dias_mes(mes, ano):
    if mes == 2 and is_leap_year(ano):
        return 29
    else:
        return dias_por_mes[mes]

# Aplicar a função para criar a nova coluna "N°dias"
df['N°dias'] = df.apply(lambda x: numero_dias_mes(x['Mês'], x['Anos']), axis=1)

# Calculando ETo mensal
df['ETo (mm/mês)'] = df.apply(lambda x: x['ETo'] * x['N°dias'], axis = 1)

# Calculando Rn mensal
df['Rn (MJ/m²mês)'] = df.apply(lambda x: x['Rn'] * x['N°dias'], axis = 1)

df['Municipio'] = df.apply(lambda x: nome_cidade, axis=1)

#selecionamdo as colunas de interesse
df = df[['Mês','Anos','Tmax (T2M_MAX)', 'Tmin (T2M_MIN)','Tmed (°C)', \
    'Chuva (mm) (PRECTOTCORR_SUM)','UR (RH2M)', 'U2 (WS2M)', \
         'Rn', 'Rn (MJ/m²mês)', \
         'N', 'n', \
         'ETo','ETo (mm/mês)','Municipio']]

# Renomear várias colunas diretamente
df.columns = ['Mês','Anos','Tmax (°C)', 'Tmin (°C)','Tmed (°C)', \
              'Chuva_total (mm)', 'UR_med (%)', 'U2_med (m/s)', \
              'Rn (MJ/m²d)', 'Rn (MJ/m²mês)', \
              'N', 'n', \
              'ETo (mm/d)','ETo (mm/mês)','Municipio']

df3 = df.groupby("Mês").mean(numeric_only=True).reset_index()
df3.drop(columns=['Anos'], inplace=True)
df3['Municipio'] = df3.apply(lambda x: nome_cidade, axis=1)

tiempo = 'Normal_Climatologica'

#para baixar os dados tire o # da última linha
from google.colab import files
df3.to_excel(f'{municipio}_{tiempo}.xlsx')
#files.download(f'{municipio}_{tiempo}.xlsx')

df3

,Mês,Tmax (°C),Tmin (°C),Tmed (°C),Chuva_total (mm),UR_med (%),U2_med (m/s),Rn (MJ/m²d),Rn (MJ/m²mês),N,n,ETo (mm/d),ETo (mm/mês),Municipio
0,1,31.231333,14.634000,22.932667,176.702000,80.752667,0.086333,16.998234,526.945258,13.457072,10.093377,5.223612,161.931962,Pato Branco
1,2,31.214667,14.239333,22.727000,157.764000,80.996667,0.090667,15.639073,442.178255,12.865385,9.811522,4.799689,135.710068,Pato Branco
2,3,30.477667,12.571000,21.524333,146.212333,80.686667,0.099333,14.120681,437.741120,12.151480,9.633320,4.285610,132.853919,Pato Branco
3,4,28.853667,7.559333,18.206500,130.381000,81.484667,0.116000,11.796878,353.906333,11.345331,10.214490,3.464417,103.932496,Pato Branco
4,5,25.796333,4.492667,15.144500,159.413333,84.138333,0.131667,9.141295,283.380155,10.694258,9.646546,2.557197,79.273092,Pato Branco
5,6,23.722000,1.829333,12.775667,145.372333,86.328000,0.122000,7.766593,232.997781,10.365959,9.570762,2.085755,62.572644,Pato Branco
6,7,23.928667,0.130333,12.029500,110.445333,84.351000,0.116333,8.713417,270.115937,10.526646,10.172649,2.335221,72.391857,Pato Branco
7,8,27.069667,2.035333,14.552500,94.317000,80.477667,0.111333,10.812958,335.201696,11.105972,10.852200,3.057102,94.770166,Pato Branco
8,9,29.258000,4.516333,16.887167,149.973333,78.704667,0.094667,12.174741,365.242216,11.888100,11.510810,3.567705,107.031154,Pato Branco
9,10,30.147333,8.766667,19.457000,224.771000,80.539000,0.086000,13.824554,428.561167,12.667053,11.638014,4.128282,127.976733,Pato Branco
